# Multiclass classification

## Learning objectives

- Understand how classification can be implemented when there are more than 2 classes
- Implement a multiclass classifier from scratch

## Binary vs Multiclass

In binary classification the output must be either `True` or `False` as we already know.

Either the example falls into this class, or it doesn't. We have seen that we can represent this by our model having a single output node whose value is forced between 0 and 1, and as such represents probability that the example belongs to the positive class.

## Multiclass

![](./images/binary-class.jpg)

In the case where we have two nodes to represent true and false, we can think about it as having trained two separate models.

Treating true and false as separate classes with separate output nodes shows us how we can extend this idea to do multiclass classification; we simply add more nodes and ensure that their values are positive and sum to one.

__Each node is a single `logit` and all of them combined are later passed to `softmax`__

![](./images/multiclass.jpg)

## Multiclass vs Multilabel

In this course we will not talk about __multilabel__ case, but:

> In multilabel problem, each label can exist simultaneously instead of exclusively like in multiclass

This might be a single vector where we have `cat` and `dog` on a picture but not a turtle:

$$
[1, 0, 1]
$$

> In multiclass, __there is always a single `1` label__, not less, not more

## Logits

Here we will also outputs logits, in case of multiclass the only change is __it will be a vector of values__. Each value in the output vector corresponds to certain class.

Let's say we would like to classify our input image into one of threes classes: `{dog=0, cat=1, turtle=2}`. Output of our model might look like this:

$$
    [-5, -3, 2]
$$

This would be a prediction of class `turtle` as it's value is highest.
When we want to get a label from this operation we use [`argmax`](https://numpy.org/doc/stable/reference/generated/numpy.argmax.html):

> argmax returns __index__ of array entry with __the highest value__

As before, we can transform them into probabilities using...

## Softmax

The **softmax function** exponentiates each value in a vector to make it positive and then divides each of them by their sum to normalise them (make them sum to 1). 

This ensures that the vector then can be interpreted as a probability distribution.

![](./images/softmax.jpg)

Real life example with real values

![](./images/softmax_example.jpg)

## Differentiating the softmax

Softmax derivative is different based on the index of element with respect to which we take derivative. 

If it is the same as the index of element we applied softmax, the derivative is the equation at the bottom, otherwise the one above it. 

![](images/softmax_deriv.jpg)

### Properties of softmax

- increasing the value of any entry decreases the value of all of the others, because the whole vector must always sum to one. 
- an increase in one input element increases it's corresponding output element exponentially whilst pushing others down, 
- this means that __it is very easy for the one largest output element to become dominant__
- because of that `softmax` is overconfident and there are ways to combat this like `label smoothing`

### What does the name "softmax" mean?

- as explained above, usually one input is near `1`, and all others close to `0`. That is, similar to the `argmax` operation mentioned previously but "soft" as it can be differentiated.
- `argmax` changes abruptly, small difference between two values make it either `0` or `1`. Softmax on the other hand changes gradually when the maximum changes

Let's implement our own softmax function, and again include a boolean flag that will return the gradient.

In [1]:
import numpy as np

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    exponentials = np.exp(x)
    return exponentials / exponentials.sum(axis=1).reshape(-1, 1)

## Stable softmax

As seen in `sigmoid` case this version also suffers from numerical instability, check out below:

In [2]:
softmax(np.array([[1000, 9, 8], [11, 12, 15]]))

<ipython-input-1-1a709be775cf>:5: RuntimeWarning: overflow encountered in exp
  exponentials = np.exp(x)
<ipython-input-1-1a709be775cf>:6: RuntimeWarning: invalid value encountered in true_divide
  return exponentials / exponentials.sum(axis=1).reshape(-1, 1)


array([[       nan, 0.        , 0.        ],
       [0.01714783, 0.04661262, 0.93623955]])

This time the result is even worse as we get `np.nan` due to overflow. Solution to the problem is to subtract maximum value from each row.

### Subtracting, what?

As `softmax` works along the horizontal axis (`1`) and all values sum to `1` the only thing that matters with the numbers in certain row is their absolute distance. 

This means we can divide __any value__ from them and still get the same results:

In [3]:
original = np.array([5, -2, 0]).reshape(1, -1)
subtracted = original - 6
softmax(original), softmax(subtracted)

(array([[9.92408247e-01, 9.04959183e-04, 6.68679417e-03]]),
 array([[9.92408247e-01, 9.04959183e-04, 6.68679417e-03]]))

### What to subtract?

There is no way to know what is the right `const` value to remove from each row. What if we have `1000` or `1_000_000`? Fortunately, we can find maximum in whole batch of data and simply subtract that.

Here is our `stable softmax`:

In [4]:
def softmax(logits):
    exps = np.exp(logits - np.max(logits, axis=1).reshape(-1, 1))
    return exps / np.sum(exps, axis=1).reshape(-1, 1)

In [5]:
softmax(np.array([[1000, 9, 8], [11, 12, 15]]))

array([[1.        , 0.        , 0.        ],
       [0.01714783, 0.04661262, 0.93623955]])

## Softmax as operation

As we have seen previously (binary case) there is no need to backpropagate through activation.

Also, same as previously, we can combine `loss` and activation into one class with improved numerical stability.

## One hot encoding

Our targets can be encoded in multiple ways. Usually, we would simply pass class numbers like this (for `5` samples):

$$
[0, 3, 1, 1, 4]
$$

We could also do that using one-hot encoding:

$$
\begin{align}
&[1, 0, 0, 0, 0]\\
&[0, 0, 0, 1, 0]\\
&[0, 1, 0, 0, 0]\\
&[0, 1, 0, 0, 0]\\
&[0, 0, 0, 0, 1]\\
\end{align}
$$

As most of the data works with the first option, we will now code how to transform `labels` into one-hot-encoding and vice versa:

In [6]:
def to_one_hot(labels, max_labels: int = None):
    if max_labels is None:
        max_labels = np.max(labels) + 1
    return np.eye(max_labels)[labels]


def to_labels(one_hot):
    return np.argmax(one_hot, axis=-1)


data = np.array([0, 1, 0, 3, 5])
to_one_hot(data)

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 1.]])

## The cross entropy loss function

An appropriate loss function to use for multiclass classification is the __cross entropy loss function__.

- It is a __generalization of binary cross entropy loss__ so it will work in binary case as well
- BCE (binary cross-entropy) is faster and more stable for binary case so __it should be created separately__ and __used separately__.

Like BCE loss, cross entropy uses the same term: __the negative natural log of the output probability__ to penalise outputs exponentially as they stray from the ground truth.

> We don't need to simultaneously push down the incorrect class probabilities and push up the correct class probabilities.

So if we focus on increasing the correct class likelihood element, then we will implicitly be decreasing the incorrect class likelihood elements.

![](images/cross_entropy_loss.jpg)

Let's implement the cross entropy loss function.

In [7]:
import aicore.ml.graph as g

class _CrossEntropyWithLogits(g.Operation):
    def forward(self, logits, targets):
        softmaxed = softmax(logits) + 1e-100
        one_hot = to_one_hot(targets)
        self.cache = (softmaxed, one_hot)
        return -np.sum(one_hot * np.log(softmaxed), axis=1)

    def backward(self, upstream_gradient):
        expanded_output = upstream_gradient.reshape(-1, 1)
        return (
            -expanded_output * np.log(self.cache[0]),
            expanded_output
            * (
                self.cache[0] * np.expand_dims(np.sum(self.cache[1], axis=1), axis=1)
                - self.cache[1]
            ),
        )


def ce_with_logits(logits, targets):
    return _CrossEntropyWithLogits()(logits, targets)

ce_with_logits(np.random.randn(100, 10), np.random.randint(low=0, high=10, size=100))

array([3.56339449, 3.28412511, 2.86014172, 3.0199154 , 0.96993968,
       3.46779906, 1.61986545, 4.28460818, 1.71629159, 2.32778261,
       2.24850963, 1.78113463, 3.8332429 , 3.30344017, 3.30277331,
       1.02874843, 2.61616417, 1.66820768, 2.87446505, 3.35076931,
       1.68441313, 2.71647111, 1.66512745, 2.520211  , 3.31368786,
       3.79938877, 3.552804  , 2.80965781, 5.97963556, 3.54317148,
       4.21934683, 1.65655944, 2.97575934, 2.15860853, 2.1733866 ,
       0.92161092, 2.99774594, 3.48824776, 3.01541179, 2.16092299,
       2.91453846, 2.66063641, 3.81874363, 3.08014895, 3.06788459,
       4.29309709, 2.96384002, 2.34852972, 3.89309769, 3.82368286,
       2.41811022, 1.93942921, 3.30421752, 4.57947271, 2.24076493,
       2.43826775, 4.28539966, 3.76299433, 3.02781664, 2.47184486,
       1.17779647, 2.43132823, 2.59588072, 2.30930862, 1.14282129,
       2.89614665, 2.14065441, 1.62074235, 2.64568602, 3.03479676,
       2.26570277, 2.69584425, 3.15914574, 2.02977169, 2.07229

## What about numerical stability?

Softmax is unlikely to return `0` with `log` for any practical case. Workarounds exist, including but not limited to:
- Add small `epsilon` constant (what we did)
- Calculate loss **only** for non-zero elements

## Multiclass logistic regression

Multiclass logistic regression, as we know, is essentially multiple linear regression joined by `cross entropy` loss function.

This finally brings us to the implementation:

In [8]:
class MulticlassLogisticRegression:
    def __init__(self, n_classes, n_features, optimizer):
        self.W = g.Parameter(np.random.randn(n_features, n_classes))
        self.b = g.Parameter(np.random.randn(n_classes))
        self.optimizer = optimizer

    def parameters(self):
        return self.W, self.b

    def predict_logits(self, X):
        return g.add(g.dot(X, self.W), self.b)

    def predict_proba(self, X):
        with g.no_grad():
            return softmax(self.predict_logits(X))

    def predict(self, X):
        with g.no_grad():
            return self.predict_logits(X) > 0

    def fit(self, X, y_true, epochs: int = 10):
        for _ in range(epochs):
            y_pred = self.predict_logits(X)
            # loss is our final node
            g.mean(ce_with_logits(y_pred, y_true))
            g.get().backward()
            self.optimizer(self.parameters())

## Testing

The scheme still applies, normalization is still important, we will just use different task. You should be pretty familiar by now of how this goes (dataset is normalized so need for this step).

Load [this dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html#sklearn.datasets.load_digits) 

In [9]:
from aicore.ml import data
from sklearn import datasets, model_selection

(X_train, y_train), (X_validation, y_validation), (X_test, y_test) = data.split(
    datasets.load_digits(return_X_y=True)
)

X_train.shape, y_train.shape

((1078, 64), (1078,))

In [10]:
def calculate_loss(model, X, y_true):
    with g.no_grad():
        y_pred = model.predict_logits(X)
        return g.mean(ce_with_logits(y_pred, y_true))


optimizer = g.optimizers.SGD(lr=1e-3)
model = MulticlassLogisticRegression(
    n_classes=10, n_features=X_train.shape[1], optimizer=optimizer
)

print(f"Training loss before fit: {calculate_loss(model, X_train, y_train)}")
print(
    f"Validation loss before fit: {calculate_loss(model, X_validation, y_validation)}"
)
print(f"Test loss before fit: {calculate_loss(model, X_validation, y_validation)}")

Training loss before fit: 111.27744990972465
Validation loss before fit: 117.39852236687295
Test loss before fit: 117.39852236687295


In [23]:
model.fit(X_train, y_train, epochs=10)

print(f"Training loss after fit: {calculate_loss(model, X_train, y_train)}")
print(
    f"Validation loss after fit: {calculate_loss(model, X_validation, y_validation)}"
)
print(f"Test loss after fit: {calculate_loss(model, X_validation, y_validation)}")

Training loss after fit: 205.90835154418
Validation loss after fit: 212.2996283512616
Test loss after fit: 212.2996283512616


## Where to use simple linear models?

We have seen how to create and use `linear models` for regression and classification.
Soon we will meet more powerful models but here is the rough summary of when one should use it in real life:
- as a baseline - gives us an overview and "starting point" for improvement
- when we want easily explaianble model. Each weight shows the impact of a factor onto our target
- when we have a lot of features (even more than data point) and we do not want to overfit on data

With experience and more models it will become increasingly clear where we should use each.

## Challenges

- What is the probability for `10` classes to be correctly predicted in __multiclass__ setting? What about __multilabel__ version?
- What loss function should we use when working with multilabel task? __Tip:__ we already presented it
- Code `cross_entropy` only by choosing elements which targets point to. Can you do it for `backward` as well?
- How does `multiclass` differ from `multilabel`? Show an example for single sample
- Try to implement alpha smoothing. Test on some datasets and check whether that helps on test
- Check out [Don't Overfit II](https://www.kaggle.com/c/dont-overfit-ii) Kaggle challenge and available solution to get a better idea when to use simple models

## Summary

- multiclass classification is multiple linear regression stacked together
- multiclass classification requires a different loss function (cross entropy)
- we can work directly on logits to take predictions by using `argmax`
- softmax is a differentiable function that turns a vector of real numbers into a probability distribution